# Srock Strategy - Multi Factor 0003

## 1. Import Library

In [1]:
import dai
import random
import pandas as pd

## 2. Prepare Strategy Data

In [2]:
sd = '2023-12-28'
ed = '2026-02-13'

In [3]:
sql_dict = {
    "factor1": {
        "sql": """
        SELECT 
            date,
            instrument, 
            -1 * m_sum(turn, 250) AS factor 
        FROM cn_stock_bar1d  
        ORDER BY date, instrument
        """,
        "weight": 0.5
    },
    "factor2": {
        "sql": """
        SELECT 
            date,
            instrument, 
            -1 * m_sum(turn, 20) AS factor 
        FROM cn_stock_bar1d  
        ORDER BY date, instrument
        """,
        "weight": 0.3
    },
    "factor3": {
        "sql": """
        SELECT 
            date,
            instrument, 
            -1 * m_sum(turn, 120) AS factor 
        FROM cn_stock_bar1d 
        ORDER BY date, instrument
        """,
        "weight": 0.2
    },
}

In [4]:
sql_list1 = []
sql_list2 = []
sql_list3 = []

for i, (factor_name, factor_data) in enumerate(sql_dict.items(), start=1):
    sql_list1.append(f"""data_{factor_name} AS ({factor_data['sql']})""")
    sql_list2.append(f"c_normalize(data_{factor_name}.factor) * {factor_data['weight']}")
    if i == 1:
        sql_list3.append(f"FROM data_{factor_name}")
    else:
        sql_list3.append(f"JOIN data_{factor_name} USING (date, instrument)")

sql_str1 = ", ".join(sql_list1) + ", "
sql_str2 = " + ".join(sql_list2)
sql_str3 = " ".join(sql_list3)

sql_combine = f"""
WITH
{sql_str1}
data_merge AS (
    SELECT
        date,
        instrument,
        {sql_str2} AS factor_new
    {sql_str3} 
)
SELECT
    date,
    instrument,
    factor_new AS factor
FROM data_merge
"""

In [5]:
sql_factor = f"""
WITH
data_alpha AS (
    {sql_combine}
),
data_alpha_origin AS (
    SELECT 
        date,
        instrument,
        factor,
    FROM data_alpha
    QUALIFY COLUMNS(*) IS NOT NULL AND factor != 'Infinity' AND factor != '-Infinity'
),
data_alpha_filter AS (
    SELECT 
        date,
        instrument,
        factor,
    FROM data_alpha_origin JOIN cn_stock_prefactors USING (date, instrument)
    WHERE amount > 0
    AND is_risk_warning = 0
    AND name NOT LIKE '%ST%'
    AND name NOT LIKE '%退%'
    AND list_days > 365
    AND (instrument LIKE '%SH' OR instrument LIKE '%SZ')
    QUALIFY COLUMNS(*) IS NOT NULL
),
data_alpha_process AS (
    SELECT 
        date,
        instrument,
        factor,
        clip(factor, c_avg(factor) - 3 * c_std(factor), c_avg(factor) + 3 * c_std(factor)) AS clipped_factor,
        c_normalize(clipped_factor) AS normalized_factor,
        c_neutralize(normalized_factor, sw2021_level1, LOG(total_market_cap)) AS neutralized_factor,
    FROM data_alpha_filter JOIN cn_stock_prefactors USING (date, instrument)
    ORDER BY date, instrument
)
SELECT 
    date, 
    instrument, 
    neutralized_factor AS factor 
FROM data_alpha_process 
ORDER BY date, factor DESC
"""

In [6]:
sql_trade = f"""
WITH
data_strategy AS (
    {sql_factor}
),
data_filter AS (
    SELECT
        date,
        instrument,
        factor AS score,
        c_rank(factor) AS score_rank,
    FROM data_strategy
    QUALIFY score_rank <= 10
),
data_date AS (
    SELECT
        date,
        instrument,
        score, 
        score_rank, 
        1 / c_sum(1) AS position, 
    FROM data_filter JOIN mldt_cn_stock_calendar_daily USING (date)
    WHERE is_month_end_trade = 1
)
SELECT *
FROM data_date
ORDER BY date, score_rank
"""

In [7]:
df_strategy = dai.query(sql_trade, filters={"date":[sd, ed]}).df()
df_strategy

,date,instrument,score,score_rank,position
0,2025-01-27,000099.SZ,-3.086832,1.0,0.1
1,2025-01-27,600611.SH,-3.018008,2.0,0.1
2,2025-01-27,601136.SH,-3.002985,3.0,0.1
3,2025-01-27,600789.SH,-2.824139,4.0,0.1
4,2025-01-27,300147.SZ,-2.747876,5.0,0.1
...,...,...,...,...,...
125,2026-01-30,002582.SZ,-3.073485,6.0,0.1
126,2026-01-30,002565.SZ,-3.055054,7.0,0.1
127,2026-01-30,301408.SZ,-3.003896,8.0,0.1
128,2026-01-30,301377.SZ,-2.995202,9.0,0.1


## 3. Backtest Strategy

In [8]:
from bigmodule import M

def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = list(df_now["instrument"])
    holding_instruments = list(context.get_account_positions().keys())

    for instrument in holding_instruments:
        if instrument not in target_instruments:
            context.order_target_percent(instrument, 0)
        
    for i, x in df_now.iterrows():
        position = 0.0 if pd.isnull(x.position) else float(x.position)
        context.order_target_percent(x.instrument, position)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df_strategy,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000  + random.uniform(0, 10),
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2026-02-18 20:19:42] [info     ] bigtrader.v34 开始运行 ..
[2026-02-18 20:19:42] [info     ] 2025-01-27, 2026-01-30, , , instruments=55
[2026-02-18 20:19:42] [info     ] bigtrader module V2.2.0
[2026-02-18 20:19:42] [info     ] bigtrader engine v0.1.0.post9+g7a244b6 2026-02-10
[2026-02-18 20:19:43] [info     ] backtest done, raw_perf_ds:dai.DataSource("_75f0596b263b4211a61e592329870bbb")


[2026-02-18 20:19:43] [info     ] bigtrader.v34 运行完成 [1.756s].
